The ENTSO-E (European Network of Transmission System Operators for Electricity) Transparency Platform, launched in 2015, serves as Europe's central hub for electricity market data. Created to fulfill EU Regulation 543/2013, it collects and publishes data from over 42 transmission system operators across Europe. The platform transformed what was once a fragmented landscape of national data sources into a unified repository, providing crucial information about power generation, consumption, and cross-border flows. For market participants and analysts, this data source is invaluable as it offers insights into the fundamental drivers of electricity prices, including generation mix, grid constraints, and demand patterns.

![Example Image](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRv8tncnUyen1EF-uNSqu9D2iw7FSClbohI4KQBGH2XPhI_7ve-W9BXXI5-vluG2t9FCLQ&usqp=CAU)

We applied for an API-Key and programmatically extract relevant information in the following.

We extracted day-ahead prices, generation mix, and cross-border flows for the year 2020. We also queried some predictions, including the generation mix for the next week, which might hint to the future day-ahead prices.

In [ ]:
# fetching data with right naming conventions

from entsoe import EntsoeRawClient, EntsoePandasClient
import pandas as pd
import numpy as np
from dotenv import load_dotenv 
import os
import datetime
import time
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

load_dotenv()
ENTSOE_API_KEY = os.getenv('ENTSOE_API_KEY')
client = EntsoePandasClient(api_key=ENTSOE_API_KEY)

start = pd.Timestamp('20150101', tz='UTC')
change_date = pd.Timestamp('20181001', tz='UTC')
end = pd.Timestamp(datetime.datetime.now(), tz='UTC')

print(os.getcwd())
out_dir = '../data'

country_code_old = 'DE_AT_LU'
country_code_new = 'DE_LU'

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10), retry=retry_if_exception_type(Exception))
def query_entsoe_data(query_func, country_code, start, end):
    try:
        df = query_func(country_code, start=start, end=end)
        
    except Exception as e:
        print(f"Error querying data: {e}")
        raise
    return df

def merge_data(query_func):
    data_old = query_entsoe_data(query_func, country_code_old, start, change_date)
    
    data_new = query_entsoe_data(query_func, country_code_new, change_date, end)

    if not isinstance(data_old, pd.DataFrame):
        data_old = data_old.to_frame()
    if not isinstance(data_new, pd.DataFrame):
        data_new = data_new.to_frame()
    
    if not data_old.empty and not data_new.empty:
        if len(data_old.columns) != len(data_new.columns):
            same_columns = list(set(data_old.columns) & set(data_new.columns))
            data_old = data_old[same_columns]
            data_new = data_new[same_columns]
        else:
            data_new.columns = data_old.columns
    df_combined = pd.concat([data_old, data_new])
    df_combined.index = df_combined.index.tz_convert('UTC')
    return df_combined

def save_df_with_timestamp(df, filename):
    df_copy = df.copy()
    df_copy.index.name = 'timestamp'
    df_copy.to_csv(filename)

# Day-ahead prices (EUR/MWh)
day_ahead_prices = merge_data(client.query_day_ahead_prices)
day_ahead_prices = day_ahead_prices.rename(columns={day_ahead_prices.columns[0]: 'day_ahead_prices_EURO'})
save_df_with_timestamp(day_ahead_prices, f'{out_dir}/day_ahead_prices.csv')
print('Day-ahead prices done')

# Load forecast (MWh)
load_forecast = merge_data(client.query_load_forecast)
load_forecast = load_forecast.rename(columns={load_forecast.columns[0]: 'E_load_forecast_MWh'})
save_df_with_timestamp(load_forecast, f'{out_dir}/load_forecast.csv')
print('Load forecast done')

# Generation forecast (MWh)
generation_forecast = merge_data(client.query_generation_forecast)
generation_forecast = generation_forecast.rename(columns={generation_forecast.columns[0]: 'E_generation_forecast_MWh'})
save_df_with_timestamp(generation_forecast, f'{out_dir}/generation_forecast.csv')
print('Generation forecast done')

# Wind and solar forecasts (MWh)
intraday_wind_solar_forecast = merge_data(client.query_intraday_wind_and_solar_forecast)
for col in intraday_wind_solar_forecast.columns:
    if 'Wind' in col:
        intraday_wind_solar_forecast = intraday_wind_solar_forecast.rename(columns={col: 'E_wind_forecast_MWh'})
    elif 'Solar' in col:
        intraday_wind_solar_forecast = intraday_wind_solar_forecast.rename(columns={col: 'E_solar_forecast_MWh'})
save_df_with_timestamp(intraday_wind_solar_forecast, f'{out_dir}/intraday_wind_solar_forecast.csv')
print('Intraday wind and solar forecast done')

# Day ahead wind and solar forecast (MWh)
day_ahead_wind_solar_forecast = merge_data(client.query_wind_and_solar_forecast)
for col in day_ahead_wind_solar_forecast.columns:
    if 'Wind' in col:
        day_ahead_wind_solar_forecast = day_ahead_wind_solar_forecast.rename(columns={col: 'E_wind_forecast_MWh'})
    elif 'Solar' in col:
        day_ahead_wind_solar_forecast = day_ahead_wind_solar_forecast.rename(columns={col: 'E_solar_forecast_MWh'})
save_df_with_timestamp(day_ahead_wind_solar_forecast, f'{out_dir}/day_ahead_wind_solar_forecast.csv')
print('Day ahead wind and solar forecast done')

# Physical crossborder flows (MWh)
physical_crossborder_flows = merge_data(lambda cc, start, end: client.query_physical_crossborder_allborders(start=start, end=end, country_code=cc, export=True))
for col in physical_crossborder_flows.columns:
    physical_crossborder_flows = physical_crossborder_flows.rename(columns={col: f'E_crossborder_{col}_actual_MWh'})
save_df_with_timestamp(physical_crossborder_flows, f'{out_dir}/physical_crossborder_flows.csv')
print('Physical crossborder flows done')


/home/jordan/uni/BTW25/final-submission/scripts
fetching data completed
fetching data completed
Day-ahead prices done


KeyboardInterrupt: 